In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import csv
import urllib.request
import copy

In [2]:
def game_player_stats(classname, loc, souploc):
    data = []
    table = souploc.findAll(('table',{'class':classname}))
    table_body = table[loc].find('tbody')
    rows = table_body.findAll('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        cols.pop(0)
        cols.pop(0)
        data.append(cols) # Get rid of empty values
    
    return data

In [3]:
def game_total_stats(classname, souploc):
    data = []
    table = souploc.findAll(('table',{'class':classname}))
    table_foot = table[2].find('tfoot')
    if not table_foot:
        table_foot = table[3].find('tfoot')
    rows = table_foot.findAll('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        if(cols[0]!="Rushed"):
            cols.pop(0)
            data.append(cols) # Get rid of empty values
    return data

In [4]:
def home_away(home,away,data):
    newdata = []
    for a,b in zip(data[home],data[away]):
        if a == '':
            a = 0
        if b == '':
            b = 0
        newdata.append(str(float(a)-float(b)))
    return newdata

In [5]:
team_names = ['Adelaide', 'Brisbane Lions', 'Carlton', 'Collingwood', 'Essendon',
 'Fremantle', 'Geelong', 'Gold Coast', 'Greater Western Sydney', 'Hawthorn', 'Melbourne',
'North Melbourne', 'Port Adelaide', 'Richmond', 'St Kilda', 'Sydney', 'West Coast', 'Western Bulldogs' ]
data2 = []
team_test = ['Adelaide']
for team in team_names:
    data2.clear()
    with open('team_lists/'+team+'.csv', newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
        
    for game in data:
        url  = game[4]
        soup = BeautifulSoup(urllib.request.urlopen(url), 'html5lib')
        data1 = game_total_stats('sortable',soup)
        if(team == game[0]):
            data1[0].extend(home_away(0,1,data1))
            data1.pop(1)
            data1[0].insert(0,str(int(game[1])-int(game[3]))) 
            data1[0].insert(0,game[2])
        else:
            data1[0].extend(home_away(1,0,data1))
            data1.pop(1)
            data1[0].insert(0,str(int(game[3])-int(game[1])))
            data1[0].insert(0,game[0])
        data1[0].insert(0,game[4][43:-5])
        print(data1)
        data2.extend(data1[:])
        
    with open('team_data/'+team+'_data.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for row in data2:
            for point in range(len(row)):
                if row[point] == '':
                    row[point] = 0
            wr.writerow(row)

[['010719990327', 'Western Bulldogs', '-13', '174', '70', '98', '272', '12', '6', '29', '21', '35', '44', '29', '32', '18', '21', '', '109', '159', '12', '9', '14', '17', '', '', '-19.0', '6.0', '-28.0', '-47.0', '-1.0', '-7.0', '7.0', '7.0', '9.0', '-12.0', '-20.0', '-5.0', '-3.0', '3.0', '-6.0', '-26.0', '-25.0', '3.0', '1.0', '-1.0', '-10.0', '0.0', '0.0']]
[['011019990403', 'Hawthorn', '4', '193', '57', '81', '274', '8', '8', '27', '20', '36', '50', '52', '31', '21', '23', '2', '139', '130', '9', '2', '20', '6', '', '', '1.0', '20.0', '-4.0', '-3.0', '0.0', '4.0', '0.0', '-2.0', '-1.0', '-3.0', '-20.0', '-9.0', '2.0', '-2.0', '2.0', '-1.0', '6.0', '7.0', '7.0', '-8.0', '4.0', '0.0', '0.0']]
[['011519990411', 'St Kilda', '15', '175', '72', '86', '261', '14', '12', '30', '29', '17', '58', '34', '28', '20', '24', '5', '124', '140', '22', '13', '26', '8', '', '', '-22.0', '-3.0', '-20.0', '-42.0', '3.0', '-3.0', '-3.0', '4.0', '-19.0', '23.0', '-8.0', '7.0', '-4.0', '4.0', '4.0', '9.0'

[['010420000319', 'Collingwood', '-11', '182', '76', '103', '285', '14', '19', '22', '31', '37', '49', '33', '32', '20', '16', '5', '116', '174', '10', '19', '34', '3', '', '', '-5.0', '-12.0', '-19.0', '-24.0', '-1.0', '-5.0', '19.0', '-4.0', '-13.0', '7.0', '11.0', '1.0', '-4.0', '4.0', '-4.0', '-10.0', '-21.0', '-3.0', '-7.0', '-7.0', '2.0', '0.0', '0.0']]
[['010920000325', 'Geelong', '-11', '213', '85', '115', '328', '14', '16', '20', '14', '31', '63', '34', '30', '22', '20', '2', '149', '174', '20', '15', '19', '4', '', '', '4.0', '-2.0', '16.0', '20.0', '-3.0', '7.0', '-1.0', '-16.0', '-9.0', '11.0', '2.0', '-1.0', '2.0', '-2.0', '-2.0', '6.0', '13.0', '5.0', '5.0', '-2.0', '-5.0', '0.0', '0.0']]
[['011820000401', 'West Coast', '-114', '226', '98', '129', '355', '29', '13', '36', '21', '48', '60', '35', '25', '21', '13', '6', '134', '221', '14', '27', '17', '7', '', '', '-53.0', '-44.0', '-29.0', '-82.0', '-19.0', '0.0', '-2.0', '1.0', '-27.0', '5.0', '21.0', '3.0', '-8.0', '8.0'

[['011320010415', 'Port Adelaide', '-65', '169', '65', '91', '260', '13', '10', '34', '49', '27', '51', '41', '25', '23', '20', '', '119', '139', '11', '17', '11', '9', '', '', '-34.0', '-9.0', '-16.0', '-50.0', '-10.0', '-5.0', '13.0', '16.0', '-5.0', '-8.0', '-1.0', '-1.0', '3.0', '-3.0', '-6.0', '-19.0', '-20.0', '1.0', '6.0', '-17.0', '-2.0', '0.0', '0.0']]
[['010320010421', 'Carlton', '9', '192', '41', '85', '277', '8', '5', '43', '43', '35', '41', '40', '28', '28', '21', '', '144', '111', '10', '5', '35', '', '', '', '16.0', '-7.0', '-30.0', '-14.0', '1.0', '3.0', '-16.0', '-6.0', '-6.0', '11.0', '9.0', '-6.0', '-7.0', '7.0', '6.0', '2.0', '-21.0', '-7.0', '-4.0', '-9.0', '2.0', '0.0', '0.0']]
[['011220010428', 'Kangaroos', '15', '188', '62', '93', '281', '12', '18', '33', '32', '20', '60', '45', '27', '20', '24', '5', '125', '152', '15', '14', '23', '4', '', '', '11.0', '-4.0', '35.0', '46.0', '2.0', '3.0', '7.0', '-1.0', '-20.0', '18.0', '11.0', '9.0', '-4.0', '4.0', '4.0', '13

[['011320020427', 'Port Adelaide', '-8', '160', '53', '96', '256', '12', '10', '39', '50', '20', '43', '42', '27', '14', '15', '1', '104', '154', '8', '7', '44', '5', '', '', '2.0', '-15.0', '1.0', '3.0', '-2.0', '4.0', '0.0', '3.0', '-4.0', '4.0', '4.0', '1.0', '-1.0', '1.0', '-4.0', '4.0', '-3.0', '-6.0', '-4.0', '12.0', '2.0', '0.0', '0.0']]
[['011420020505', 'Richmond', '32', '164', '79', '109', '273', '11', '7', '28', '48', '34', '41', '31', '37', '10', '14', '', '107', '172', '15', '12', '49', '10', '', '', '32.0', '-1.0', '22.0', '54.0', '3.0', '14.0', '5.0', '10.0', '-9.0', '19.0', '7.0', '-5.0', '4.0', '-4.0', '6.0', '21.0', '31.0', '-5.0', '2.0', '1.0', '9.0', '0.0', '0.0']]
[['010820020511', 'Fremantle', '29', '187', '67', '146', '333', '18', '10', '42', '43', '21', '58', '43', '25', '15', '13', '5', '124', '212', '7', '10', '42', '15', '', '', '18.0', '1.0', '6.0', '24.0', '5.0', '-1.0', '21.0', '6.0', '-13.0', '19.0', '8.0', '-4.0', '2.0', '-2.0', '4.0', '10.0', '17.0', '-

[['010420030510', 'Collingwood', '-5', '180', '51', '123', '303', '16', '13', '60', '55', '24', '50', '49', '22', '11', '6', '3', '140', '155', '8', '10', '24', '4', '8', '', '17.0', '-3.0', '39.0', '56.0', '-2.0', '7.0', '33.0', '5.0', '-4.0', '6.0', '7.0', '-8.0', '5.0', '-5.0', '0.0', '28.0', '32.0', '-2.0', '0.0', '-24.0', '3.0', '-3.0', '0.0']]
[['011920030517', 'Brisbane Lions', '-18', '176', '92', '102', '278', '16', '11', '30', '40', '30', '55', '37', '33', '11', '9', '5', '107', '168', '13', '14', '48', '6', '8', '', '14.0', '-13.0', '24.0', '38.0', '-4.0', '6.0', '17.0', '-14.0', '1.0', '-5.0', '4.0', '2.0', '-2.0', '2.0', '-4.0', '21.0', '20.0', '-3.0', '-2.0', '-12.0', '-4.0', '0.0', '0.0']]
[['010720030525', 'Western Bulldogs', '16', '204', '97', '136', '340', '16', '13', '51', '36', '29', '67', '43', '34', '9', '10', '3', '127', '211', '9', '14', '38', '4', '8', '', '32.0', '22.0', '-6.0', '26.0', '2.0', '4.0', '37.0', '-8.0', '-17.0', '19.0', '14.0', '1.0', '-1.0', '1.0'

...

[['030720150704', 'Carlton', '11', '224', '108', '164', '388', '9', '10', '49', '65', '36', '55', '49', '40', '15', '8', '6', '140', '244', '8', '11', '41', '10', '7', '', '24.0', '10.0', '23.0', '47.0', '2.0', '-1.0', '2.0', '-6.0', '-6.0', '6.0', '17.0', '-6.0', '7.0', '-7.0', '6.0', '19.0', '32.0', '-6.0', '3.0', '-3.0', '2.0', '3.0', '0.0']]
[['072020150711', 'Gold Coast', '22', '237', '106', '155', '392', '14', '11', '45', '51', '37', '58', '33', '60', '17', '23', '5', '151', '246', '11', '10', '39', '10', '11', '', '42.0', '33.0', '36.0', '78.0', '3.0', '4.0', '10.0', '-18.0', '-2.0', '7.0', '-10.0', '2.0', '-6.0', '6.0', '4.0', '7.0', '79.0', '-3.0', '-2.0', '-11.0', '0.0', '2.0', '0.0']]
[['070920150718', 'Geelong', '-8', '209', '106', '184', '393', '10', '12', '41', '86', '41', '49', '40', '42', '14', '9', '3', '153', '245', '19', '14', '50', '4', '8', '', '-27.0', '-33.0', '-1.0', '-28.0', '-1.0', '-2.0', '16.0', '7.0', '-7.0', '2.0', '1.0', '4.0', '-5.0', '5.0', '0.0', '-22.

[['071620160702', 'Sydney', '4', '206', '88', '185', '391', '11', '13', '33', '81', '42', '53', '32', '50', '16', '17', '5', '154', '231', '21', '14', '37', '4', '8', '', '8.0', '-7.0', '-7.0', '1.0', '2.0', '-8.0', '16.0', '-1.0', '-5.0', '3.0', '9.0', '0.0', '1.0', '-1.0', '-4.0', '1.0', '10.0', '-8.0', '-6.0', '17.0', '6.0', '1.0', '0.0']]
[['071420160709', 'Richmond', '10', '208', '107', '188', '396', '12', '13', '26', '83', '31', '48', '38', '39', '23', '15', '4', '127', '268', '5', '7', '28', '2', '6', '', '-12.0', '-14.0', '36.0', '24.0', '1.0', '4.0', '-30.0', '7.0', '-2.0', '3.0', '4.0', '-17.0', '8.0', '-8.0', '2.0', '-7.0', '31.0', '-8.0', '-6.0', '-10.0', '-2.0', '-1.0', '0.0']]
[['072020160716', 'Gold Coast', '48', '250', '124', '217', '467', '15', '17', '24', '56', '29', '57', '36', '39', '23', '15', '6', '132', '334', '7', '18', '38', '10', '9', '', '53.0', '19.0', '73.0', '126.0', '7.0', '6.0', '-11.0', '9.0', '-9.0', '18.0', '10.0', '-18.0', '8.0', '-8.0', '6.0', '17.0

[['010720170707', 'Adelaide', '-59', '248', '127', '148', '396', '16', '8', '41', '84', '47', '54', '21', '49', '18', '23', '6', '151', '256', '20', '16', '69', '5', '12', '', '-38.0', '-67.0', '8.0', '-30.0', '-11.0', '7.0', '-18.0', '-9.0', '-14.0', '3.0', '21.0', '5.0', '5.0', '-5.0', '-6.0', '6.0', '-53.0', '-16.0', '-7.0', '-20.0', '-5.0', '-9.0', '0.0']]
[['030720170716', 'Carlton', '20', '224', '113', '136', '360', '9', '8', '54', '58', '36', '40', '32', '51', '21', '19', '1', '128', '235', '9', '14', '35', '3', '9', '', '-11.0', '-16.0', '79.0', '68.0', '3.0', '2.0', '-35.0', '3.0', '-7.0', '10.0', '1.0', '6.0', '-2.0', '2.0', '4.0', '10.0', '46.0', '6.0', '-3.0', '11.0', '8.0', '0.0', '0.0']]
[['072020170722', 'Gold Coast', '54', '232', '119', '176', '408', '16', '14', '27', '59', '30', '63', '30', '46', '24', '13', '6', '139', '274', '12', '19', '42', '18', '14', '', '42.0', '25.0', '27.0', '69.0', '8.0', '6.0', '-17.0', '-1.0', '-12.0', '22.0', '-1.0', '-18.0', '11.0', '-11.

[['071120180714', 'Melbourne', '-50', '219', '102', '168', '387', '18', '11', '51', '63', '46', '52', '31', '69', '16', '21', '6', '152', '239', '16', '12', '59', '4', '15', '', '-17.0', '-32.0', '53.0', '36.0', '-8.0', '-2.0', '-27.0', '5.0', '-16.0', '5.0', '7.0', '-15.0', '5.0', '-5.0', '-6.0', '7.0', '31.0', '-5.0', '-3.0', '1.0', '4.0', '-7.0', '0.0']]
[['071820180722', 'West Coast', '-54', '251', '131', '163', '414', '14', '16', '39', '67', '35', '52', '34', '51', '20', '20', '6', '127', '292', '15', '19', '39', '3', '10', '', '-83.0', '-56.0', '20.0', '-63.0', '-8.0', '-6.0', '-9.0', '-2.0', '0.0', '-8.0', '-4.0', '2.0', '0.0', '0.0', '-6.0', '-10.0', '-58.0', '-11.0', '-10.0', '16.0', '1.0', '-5.0', '0.0']]
[['071320180729', 'Port Adelaide', '-44', '191', '40', '147', '338', '5', '4', '43', '83', '37', '49', '50', '72', '23', '27', '1', '163', '165', '6', '5', '64', '2', '3', '', '-31.0', '-21.0', '25.0', '-6.0', '-6.0', '-8.0', '-4.0', '-3.0', '-3.0', '-9.0', '8.0', '12.0', '-

[['070820190728', 'Fremantle', '47', '248', '112', '169', '417', '16', '17', '14', '41', '34', '59', '35', '48', '22', '20', '5', '139', '276', '12', '13', '60', '11', '13', '', '52.0', '15.0', '29.0', '81.0', '7.0', '5.0', '-42.0', '-5.0', '-8.0', '15.0', '3.0', '-2.0', '2.0', '-2.0', '4.0', '22.0', '60.0', '-3.0', '4.0', '10.0', '8.0', '7.0', '0.0']]
[['071920190804', 'Brisbane Lions', '-18', '220', '96', '101', '321', '14', '14', '51', '72', '47', '54', '32', '47', '14', '12', '2', '150', '164', '18', '19', '64', '4', '10', '', '5.0', '1.0', '69.0', '74.0', '-3.0', '0.0', '-21.0', '-1.0', '-8.0', '5.0', '10.0', '3.0', '-2.0', '2.0', '2.0', '-12.0', '82.0', '-5.0', '-5.0', '-13.0', '0.0', '1.0', '0.0']]
[['050720190810', 'Essendon', '104', '195', '80', '125', '320', '4', '9', '43', '65', '44', '37', '33', '60', '23', '28', '', '126', '189', '4', '4', '54', '7', '1', '', '57.0', '26.0', '68.0', '125.0', '17.0', '2.0', '-19.0', '1.0', '-11.0', '29.0', '-3.0', '-7.0', '5.0', '-5.0', '6.